# 理解程式碼內容與生成程式碼

使用 gpt-35-turbo 與 gpt-3 支援的 Completion API 來進行自動完成

In [5]:
import os
import openai
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = "2023-05-15"

API_KEY = os.getenv('OPENAI_API_KEY','<您的 API 鍵值>').strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","https://<您的 Azure OpenAI 資源名稱>.openai.azure.com/").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

response = openai.Completion.create(
  engine=os.getenv('DEPLOYMENT_NAME','gpt-35-turbo'),
  prompt="### Postgres SQL tables, with their properties:\n" \
         "#\n" \
         "# Employee(id, name, department_id) \n" \
         "# Department(id, name, address)\n" \
         "# Salary_Payments(id, employee_id, amount, date) \n" \
         "### " \
         "查詢列出過去 3 個月僱用超過 10 名員工的部門名稱 \n"
         "query: SELECT",
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])

print(response.choices[0].text)

 Department.name FROM Department 
        JOIN Employee ON Department.id = Employee.department_id 
        JOIN Salary_Payments ON Employee.id = Salary_Payments.employee_id 
        WHERE Salary_Payments.date >= NOW() - INTERVAL '3 months' 
        GROUP BY Department.name 
        HAVING COUNT(DISTINCT Employee.id) > 10


嘗試使用 gpt-35-turbo 與 gpt-4 方支援的 ChatCompletion API 來進行自動完成

In [11]:
response  = openai.ChatCompletion.create( 
  engine = os.getenv('DEPLOYMENT_NAME','gpt-35-turbo'),  
  messages = [ 
    {'role': 'user', 'content': "告訴我以下這個 SQL 是什麼意思??\n"
      'Code:' \
      "SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n" \
      'Answer:'    
    }
  ],
  temperature=0,
  max_tokens=250
)

print(response['choices'][0]['message']['content']) 

這個 SQL 查詢的意思是：從 Department 表格和 Employee 表格中選擇部門名稱（d.name），並且只選擇那些有超過 10 個員工的部門。同時，這些員工必須在過去 3 個月內收到薪水，薪水資訊儲存在 Salary_Payments 表格中。
